# Speeding up Finance Functions with Numba and Cython

In [1]:
%load_ext cythonmagic

In [10]:
import numpy as np
import scipy.stats as si
import sympy as sy
import sympy.statistics as systats

In [11]:
from numba import double
from numba.decorators import jit, autojit

In [14]:
def euro_vanilla_call(S, K, T, r, sigma):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of underlying asset

    S = float(S)
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    call = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    
    return call

In [16]:
%timeit euro_vanilla_call(100, 50, 1, 0.05, 0.25)

1000 loops, best of 3: 339 µs per loop


In [18]:
euro_call = autojit(euro_vanilla_call)

%timeit euro_call(100, 50, 1, 0.05, 0.25)

1 loops, best of 3: 463 µs per loop


In [38]:
@autojit
def euro_call_numba(S, K, T, r, sigma):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of underlying asset

    S = float(S)
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    call = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    
    return call

In [39]:
%timeit euro_call_numba(100, 50, 1, 0.05, 0.25)

1 loops, best of 3: 245 µs per loop


In [44]:
%%cython

cimport cython
from libc.math cimport sqrt, log, exp, erf, pow

@cython.cdivision(True)
cdef double norm_cdf(double x):
    return 0.5 * (1 + erf(x / sqrt(2.0)))

@cython.cdivision(True)
def euro_call_cy(double S, double K, double T, double r, double sigma):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of underlying asset
    
    cdef double d1, d2, call
    
    d1 = (log(S / K) + (r + 0.5 * pow(sigma, 2) * T) / (sigma * sqrt(T)))
    d2 = (log(S / K) + (r - 0.5 * pow(sigma, 2) * T) / (sigma * sqrt(T)))
    
    call = (S * norm_cdf(d1) - K * exp(-r * T) * norm_cdf(d2))
    
    return call

In [45]:
%timeit euro_call_cy(100, 50, 1, 0.05, 0.25)

1000000 loops, best of 3: 449 ns per loop


In [ ]:
from IPython.core.display import HTML
import urllib2
HTML(urllib2.urlopen('https://goo.gl/gO9Vah').read())